In [1]:
import altair as alt
import pandas as pd
import sys
sys.path.append('figure_scripts/')
import plot_scripts as ps

In [2]:
RUN_TYPES = [
    'noN_filtered_15_85_human_revel_mvp_noWL', ##default
    'noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL', ##default + AM + Splice
    'noN_filtered_15_85_human_cadd_revel_mvp_alphaM_spliceAI_noWL' ,#default + AM + Splice + CADD
    'noN_filtered_15_85_human_PP_MT_SIFT_noWL', #PP + MT + SIFT
    'noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_PP_MT_SIFT_noWL' # all but CADD

]

In [3]:
success_table=pd.read_csv('supp_fig7_input.txt' , sep='\t')


## A: Combinations of path predictors

In [4]:
combo_df,denom = ps.create_df(535, RUN_TYPES, success_table)
combo_plot=ps.create_plot(denom, combo_df, RUN_TYPES, 'set2', False,RUN_TYPES).properties(
        width=500,
        height=300)
combo_plot

noN_filtered_15_85_human_revel_mvp_noWL highest rank: 76.0 max%: 96.73202614379085 148
noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL highest rank: 83.0 max%: 96.73202614379085 148
noN_filtered_15_85_human_cadd_revel_mvp_alphaM_spliceAI_noWL highest rank: 190.0 max%: 96.07843137254902 147
noN_filtered_15_85_human_PP_MT_SIFT_noWL highest rank: 142.0 max%: 96.07843137254902 147
noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_PP_MT_SIFT_noWL highest rank: 142.0 max%: 96.07843137254902 147


alt.Chart(...)

## B-E: Distribution of score values

In [7]:
order=['NONE', 'ALPHA_MISSENSE', 'MVP', 'REVEL', 'SPLICE_AI', 'CADD','MUTATION_TASTER', 'POLYPHEN', 'SIFT','REMM', 'Not_Prioritized']


In [8]:
default = ps.create_score_plot('noN_filtered_15_85_human_revel_mvp_noWL', success_table, order)
best_params = ps.create_score_plot('noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL', success_table, order)
old_tools = ps.create_score_plot('noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_PP_MT_SIFT_noWL', success_table, order)
with_cadd = ps.create_score_plot('noN_filtered_15_85_human_cadd_revel_mvp_alphaM_spliceAI_noWL', success_table, order)


In [13]:
score_panel = alt.vconcat(alt.hconcat(default, best_params), alt.hconcat(old_tools,with_cadd))


## F-G: Distribution of maximum pathogenicity SOURCE

In [14]:
all_results = pd.read_csv('ES_scores_allresults.tsv', sep='\t')

all_results['variant'] = [x.split('_')[0] for x in all_results['ID']]
#success_table['variant'] = [x.split('_')[0] for x in success_table['variant_exomiser_form']]
import numpy as np
all_results['id_variant'] = np.array(all_results['Patient_ID']) + '_' + np.array(all_results['variant'])
success_table['UDN_ID'] =[x.split('_')[0] for x in list(success_table['ID'])]
success_table['id_variant'] = np.array(success_table['UDN_ID']) + '_' + np.array(success_table['genomic_coordinates_hg38'])
all_results.fillna('NONE', inplace=True)


/tmp/ipykernel_406417/4272868802.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NONE' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  all_results.fillna('NONE', inplace=True)


In [15]:
non_diagnostic = all_results[~all_results['id_variant'].isin(list(success_table['id_variant']))]
diagnostic = all_results[all_results['id_variant'].isin(list(success_table['id_variant']))]


In [16]:
data = []
for RUN_TYPE in RUN_TYPES:
    results = non_diagnostic[non_diagnostic['RunType']==RUN_TYPE]
    results_noNone = results[results['MAX_PATH'] != 'NONE']
    print(RUN_TYPE, len(results_noNone))

    for path_source in order:
        path = results_noNone[results_noNone['MAX_PATH_SOURCE']==path_source]
        data.append([RUN_TYPE, path_source, len(path), len(path)/len(results_noNone), len(results)])
        df = pd.DataFrame(data, columns=['RunType', 'MAX_PATH_SOURCE', 'num', 'percentage', 'all_runtype'])
        
base=alt.Chart(df,title='Max path sources for NONDIAGNOSTIC variants')       
bars = base.mark_bar().encode(
    alt.X('RunType:N', sort=RUN_TYPES),
    alt.Y('num:Q', title='Number of variants'),
    color=alt.Color('MAX_PATH_SOURCE:N', sort=order, scale=alt.Scale(scheme='tableau20', domain=order)),
    tooltip=['RunType:N','MAX_PATH_SOURCE', 'num:Q', 'percentage']

).properties(width=400, height=200)
text = base.mark_point(
    align='left',
    baseline='middle',
    #dx=-5,
    #dy=-10,
    strokeDash=[5,1],
    size=5000
  
).encode(
    y=alt.Y('all_runtype:Q').axis(offset=5, domainOpacity=0),
    x=alt.X('RunType:N', sort=RUN_TYPES), 
    color=alt.value('black'),
    shape=alt.Shape('RunType:N',scale = alt.Scale(range=["stroke"])),
    text='all_runtype:Q'
)
all_variants = alt.layer(bars, text).resolve_scale(color='independent')

noN_filtered_15_85_human_revel_mvp_noWL 8953
noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL 10425
noN_filtered_15_85_human_cadd_revel_mvp_alphaM_spliceAI_noWL 19405
noN_filtered_15_85_human_PP_MT_SIFT_noWL 9380
noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_PP_MT_SIFT_noWL 10947


In [17]:
data = []
for RUN_TYPE in RUN_TYPES:
    results = success_table[success_table['Variant_Level_noMOI_' + str(RUN_TYPE)]=='Variant_Present_noMOI']
    results_noNone = results[results['Variant_Level_noMOI_maxPathSource_'+str(RUN_TYPE)] != 'NONE']
    max_paths = list(results_noNone['Variant_Level_noMOI_maxPathSource_'+str(RUN_TYPE)])
    print(RUN_TYPE, len(results))

    for path_source in order:
        num_path = max_paths.count(path_source)
      #  print(path_source, len(path))
        data.append([RUN_TYPE, path_source, num_path, num_path/len(max_paths), len(results)])

df = pd.DataFrame(data, columns=['RunType', 'MAX_PATH_SOURCE', 'num', 'percentage', 'all_runtype'])

base = alt.Chart(df,title='Max path sources for DIAGNOSTIC variants')
bars =base.mark_bar().encode(
    alt.X('RunType:N', sort=RUN_TYPES),
    alt.Y('num:Q', title='Number of variants (max=296)'),
    color=alt.Color('MAX_PATH_SOURCE:N', sort=order, scale=alt.Scale(scheme='tableau20', domain=order))

).properties(width=400, height=200)
text = base.mark_point(
    align='left',
    baseline='middle',
    dx=-5,
    dy=-10,
    color='black',
    strokeDash=[5,1],
    size=5000
).encode(
    y=alt.Y('all_runtype:Q').axis(offset=5, domainOpacity=0),
    x=alt.X('RunType:N', sort=RUN_TYPES), 
    color=alt.value('black'),
    shape=alt.Shape('RunType:N',scale = alt.Scale(range=["stroke"])),
    text='all_runtype:Q'
)

text2 = base.mark_text(
    align='left',
    baseline='middle',
    dx=-5,
    dy=-5,
    color='black',

).encode(
    y=alt.Y('all_runtype:Q').axis(offset=5, domainOpacity=0),
    x=alt.X('RunType:N', sort=RUN_TYPES), 
    color=alt.value('black'),
    text='all_runtype:Q'
)

diagnostic_variants = alt.layer(bars, text).resolve_scale(color='independent')
  

noN_filtered_15_85_human_revel_mvp_noWL 148
noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL 148
noN_filtered_15_85_human_cadd_revel_mvp_alphaM_spliceAI_noWL 147
noN_filtered_15_85_human_PP_MT_SIFT_noWL 147
noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_PP_MT_SIFT_noWL 147


In [18]:
variant_bars = alt.vconcat(all_variants, diagnostic_variants)
variant_bars

alt.VConcatChart(...)

## H-I: Distribution of maximum pathogenicity SCORES

In [19]:
from pathlib import Path

tempdir = Path("./temp")
tempdir.mkdir(exist_ok=True)

source = tempdir / "data.json"
non_diagnostic[~non_diagnostic['MAX_PATH'].isin(['NONE' , 'Not_Prioritized'])].to_json(source, orient="records")

In [20]:
for run_type in RUN_TYPES:
    scores = non_diagnostic[non_diagnostic['RunType']==run_type]
    scores = scores[scores['MAX_PATH']!='NONE']
    print(run_type,len(scores))

noN_filtered_15_85_human_revel_mvp_noWL 8953
noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL 10425
noN_filtered_15_85_human_cadd_revel_mvp_alphaM_spliceAI_noWL 19405
noN_filtered_15_85_human_PP_MT_SIFT_noWL 9380
noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_PP_MT_SIFT_noWL 10947


In [21]:
all_variants_max_path =alt.Chart(str(source), title='Variant path scores of NONDIAGNOSTIC variants').mark_boxplot().encode(
    alt.X("MAX_PATH:Q").scale(zero=False),
    alt.Y('RunType:N',sort=RUN_TYPES,title=None),#,axis=alt.Axis(labels=False)),
    color=alt.Color('RunType:N', scale=alt.Scale(scheme='set2', domain=RUN_TYPES))).properties(width=500)

In [22]:
data = []
for run_type in RUN_TYPES:
    variant_scores = list(success_table['Variant_Level_noMOI_variant_'+run_type])
    for score in variant_scores:
        data.append([score, run_type])
df= pd.DataFrame(data, columns=['EXOMISER_VARIANT_SCORE', 'RunType'])


data2 = []
for run_type in RUN_TYPES:
    variant_scores = list(success_table['Variant_Level_noMOI_maxPath_'+run_type])
    for score in variant_scores:
        data2.append([score, run_type])
df2= pd.DataFrame(data2, columns=['MAX_PATH', 'RunType'])

In [23]:
diagnostic_only_maxpath =alt.Chart(df2, title='Pathogenicity score of DIAGNOSTIC variants only').mark_boxplot().encode(
    alt.X("MAX_PATH:Q").scale(zero=False,domain=[0,1]),
    alt.Y('RunType:N',sort=RUN_TYPES,title=None),
    color=alt.Color('RunType:N', scale=alt.Scale(scheme='set2', domain=RUN_TYPES))).properties(width=500)



In [24]:
score_distrib = alt.vconcat(diagnostic_only_maxpath, all_variants_max_path)


In [26]:
alt.hconcat(alt.vconcat(combo_plot, score_distrib), alt.vconcat(score_panel,variant_bars)).configure_axis(grid=True,
    labelPadding= 5,
    labelLimit=0,
    labelFontSize=15, 
    titleFontSize=15, labelFont='arial', tickSize=8).configure_legend(
        labelLimit=0,labelFontSize=15, titleFontSize=15, labelFont='arial')

alt.HConcatChart(...)